In [ ]:
import asyncio
from random import random

In [ ]:
async def echo(index: int):
    await asyncio.sleep(0.1)
    return index


async def echo_random_latency(index: int):
    await asyncio.sleep(random())
    return index

# Test Executor 

In [ ]:
from ragas.executor import is_event_loop_running, as_completed

In [ ]:
assert is_event_loop_running() is True, "is_event_loop_running() returned False"

In [ ]:
async def _run():
    results = []
    for t in await as_completed([echo(1), echo(2), echo(3)], 3):
        r = await t
        results.append(r)
    return results


results = await _run()

expected = [1, 2, 3]
assert results == expected, f"got: {results}, expected: {expected}"

## Test Executor

In [ ]:
from ragas.executor import Executor

In [ ]:
# test order of results when they should return in submission order
executor = Executor(raise_exceptions=True)
for i in range(10):
    executor.submit(echo, i, name=f"echo_{i}")

results = executor.results()
assert results == list(range(10))

In [ ]:
# test order of results when may return unordered
executor = Executor(batch_size=None)

# add jobs to the executor
for i in range(10):
    executor.submit(echo_random_latency, i, name=f"echo_order_{i}")

# Act
results = executor.results()
# Assert
assert results == list(range(10))

In [ ]:
# Test output order; batching
executor = Executor(batch_size=3)

# add jobs to the executor
for i in range(10):
    executor.submit(echo_random_latency, i, name=f"echo_order_{i}")

# Act
results = executor.results()
# Assert
assert results == list(range(10))

In [ ]:
# Test no progress
executor = Executor(show_progress=False)

# add jobs to the executor
for i in range(10):
    executor.submit(echo_random_latency, i, name=f"echo_order_{i}")

# Act
results = executor.results()
# Assert
assert results == list(range(10))

In [ ]:
# Test multiple submission sets
executor = Executor(raise_exceptions=True)
for i in range(1000):
    executor.submit(asyncio.sleep, 0.01)

results = executor.results()
assert results, "Results should be list of None"

for i in range(1000):
    executor.submit(asyncio.sleep, 0.01)

results = executor.results()
assert results, "Results should be list of None"

# Test Metric

In [ ]:
from ragas.metrics.base import Metric


class FakeMetric(Metric):
    name = "fake_metric"
    _required_columns = ("user_input", "response")

    def init(self):
        pass

    async def _ascore(self, row, callbacks) -> float:
        return 0


fm = FakeMetric()

In [ ]:
score = fm.score({"user_input": "a", "response": "b"})
assert score == 0

# Test run_async_tasks

In [ ]:
from ragas.async_utils import run_async_tasks

In [ ]:
# run tasks unbatched
tasks = [echo_random_latency(i) for i in range(10)]
results = run_async_tasks(tasks, batch_size=None, show_progress=True)
# Assert
assert sorted(results) == list(range(10))

In [ ]:
# run tasks batched
tasks = [echo_random_latency(i) for i in range(10)]
results = run_async_tasks(tasks, batch_size=3, show_progress=True)
# Assert
assert sorted(results) == list(range(10))

In [9]:
# Test no progress
tasks = [echo_random_latency(i) for i in range(10)]
results = run_async_tasks(tasks, batch_size=3, show_progress=False)
# Assert
assert sorted(results) == list(range(10))